In [1]:
import pandas as pd
import numpy as np
import re, string
import json
import tweepy
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

In [2]:
#Loading the API keys
def loadKeys(key_file):
    with open(key_file) as f:
        key_dict = json.load(f)
    return key_dict['api_key'], key_dict['api_secret'], key_dict['token'], key_dict['token_secret']

In [3]:
#Setting access tokens
# KEY_FILE = 'twitterkeys.json'
# api_key, api_secret, token, token_secret = loadKeys(KEY_FILE)
api_key = 'IN2XT1sKUhwQLlTpK5MNLq99J'
api_secret = '8D9Nl9lcB3s9zvnUPQmZNxD4z4GHzF1lZUKsLrHv05DrNuTIea'
token = '587635645-WA4IyQmpu3iAxHUiHz9loVxuBKURD5nNikqs1OtH'
token_secret = 'CCEWXsPq4qYR92dYrcakbgWMJeSdHlOCazr10KyDEklfL'
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(token, token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
accounts = ['@satyanadella', '@jennylayfluffy', '@BradSmi', '@Microsoft', '@MSFTEnable', '@MSFTnews', '@ChuckRobbins', '@WeAreCisco', '@Cisco', '@PGelsinger', '@AmyLWarner1', '@WeAreIntel', '@intel']

In [5]:
# # Authorize our Twitter credentials
# auth = tweepy.OAuthHandler(api_key, api_secret)
# auth.set_access_token(token, token_secret)
# api = tweepy.API(auth)
# tweets = api.user_timeline(screen_name='@MSFTEnable', 
#                            # 200 is the maximum allowed count, 
#                            count = 10,
#                            include_rts = True,
#                            # Necessary to keep full_text 
#                            # otherwise only the first 140 words are extracted
#                            tweet_mode = 'extended'
#                            )

In [6]:
# tweets

In [7]:
dt, text, source, links = [], [], [], []

In [8]:
keywords = ['disabled people', 'disabled employee', 'family friendly benefits', 'parental benefits', 'leave benefits', 'health benefits', 'disability benefits', 'employee benefits', 'disability', 'disabled person', 'disabilities', 'autism', 'autistic', 
            'accessibility', 'accessible', 'erg', 'employee resource group', 'inclusivity', 'inclusion',
            'inclusive', 'visually impaired', 'hearing impaired' 'deaf',
            'discriminate', 'discriminating', 'discrimination', 'equal employment', 'equal opportunity', 'accommodation', 'accommodations', 'accommodate',
            'accommodating', 'diversity', 'diverse', 'ndeam', 'national disability employment awareness month', 'a11y', 'abilities', 'paralysed', 'paralyzed', 
            'paralysis', 'blind', 'blindness', 'culture', 'representation', 'dei', 'd&i', 'wheelchair', 
            'neurodiverse', 'deafness', 'hire', 'hired', 'hiring', 'open role', 'open roles', 'job shortage', 'dream job', 'a11yjob', 'a11yjobs', 'new job', 'job skills' 'job search', 'job hunt', 'jobs', 'job seekers', 'job opportunity', 'job opportunities',
            'interview', 'employment', 'employer', 'recruitment', 'recruiting', 'recruit']
categories = {
    'disability': ['disabled people', 'disabled employee', 'disability', 'disabled person', 'disabilities', 'autism', 'autistic',
                   'accessibility', 'accessible', 'visually impaired', 'hearing impaired', 
                   'deaf', 'accommodation', 'accommodations', 'accommodate', 'accommodating', 'paralysis', 
                    'blind', 'blindness','ndeam', 'national disability employment awareness month', 
                   'a11y', 'abilities', 'paralysed', 'paralyzed', 'wheelchair', 
                    'neurodiverse', 'deafness'
                  ],
    'hiring': ['family friendly benefits', 'parental benefits', 'leave benefits', 'health benefits', 'disability benefits', 'employee benefits', 'equal employment', 'equal opportunity', 'open role', 'open roles',
               'hire', 'hired', 'hiring', 'dream job', 'job shortage', 'new job', 'job skills' 'job search', 'job hunt', 'jobs', 
               'job seekers', 'job opportunity', 'job opportunities', 'interview', 'employment', 'employer', 'recruitment', 'recruiting', 'recruit'],
    'culture': ['erg', 'employee resource group', 'resource group', 'inclusivity', 'inclusion', 'inclusive', 'discriminate', 
                'discriminating', 'diversity', 'diverse', 'culture', 'representation', 'dei']
}

In [9]:
# keywords = ['disabled', 'disabilit', 'autism', 'autistic', 'accessibility', 'accessible', 
#                    'visually impaired', 'visual impaired', 'deaf', 'accommodat', 'paralysis', 'paralyz', 'paralys', 'blind', 'a11y',
#                    'ndeam', 'national disability employment awareness month', 'abilities', 'paralysed', 'paralyzed',
#                    'wheelchair', 'hearing', 'neurodiverse', 'benefits', 'equal employment', 'equal opportunity', 'hire', 'hiring', 
#             'jobs', 'my job', 'his job', 'her job', 
#                'their job', 'this job', 'that job', 'dream job', 'job seeker', 'job opportunit', 
#                'job market', 'job alert', 'job skill', 'interview', 'employment', 'employer', 'recruit', 
#            'erg', 'employee resource group', 'inclusiv', 'inclusion', 'discriminat', 
#                 'diversity', 'diverse', 'culture', 'representation', 'dei']
# categories = {
#     'disability': ['disabled', 'disabilit', 'autism', 'autistic', 'accessibility', 'accessible', 
#                    'visually impaired', 'visual impaired', 'deaf', 'accommodat', 'paralysis', 'paralyz', 'paralys', 'blind', 'a11y',
#                    'ndeam', 'national disability employment awareness month', 'abilities', 'paralysed', 'paralyzed',
#                    'wheelchair', 'hearing', 'neurodiverse'
#                   ],
#     'hiring': ['benefits', 'equal employment', 'equal opportunity', 'hire', 'hiring', 'jobs', 'my job', 'his job', 'her job', 
#                'their job', 'this job', 'that job', 'dream job', 'job seeker', 'job opportunit', 
#                'job market', 'job alert', 'job skill', 'interview', 'employment', 'employer', 'recruit'],
#     'culture': ['erg', 'employee resource group', 'inclusiv', 'inclusion', 'discriminat', 
#                 'diversity', 'diverse', 'culture', 'representation', 'dei']
# }

In [26]:
def clean(str):
    str = str.replace('d&i', 'dei')
    str = re.sub(r'^https?:\/\/.*[\r\n]*', '', str.lower(), flags=re.MULTILINE)
    str = re.sub(r'[^\w\s]', '', str).strip()
    #str = ''.join(i for i in str if ord(i) not in range(48, 58))
    return str
    
def keys(str):
    key = ''
    for i in keywords:
        if re.search(r'\b' + i + r'\b', str):
            key +=  i + ' ' 
    return ' '.join(sorted(set(key.split())))

def isRelevant(str):
    for i in keywords:
        if re.search(r'\b' + i + r'\b', str):
            return 1 
    return 0

def tags(str):
    labels = ''
    for i in categories['disability']:
        if re.search(r'\b' + i + r'\b', str):
            labels += 'disability' + ' '
            break
    for i in categories['culture']:
        if re.search(r'\b' + i + r'\b', str):
            labels += 'culture' + ' '
            break
    for i in categories['hiring']:
        if re.search(r'\b' + i + r'\b', str):
            labels += 'hiring' + ' '
            break
    return ' '.join(sorted(set(labels.split())))

def company(str):
    if str == 'satyanadella' or str == 'jennylayfluffy' or str == 'BradSmi' or str == 'MSFTEnable' or str == 'MSFTNews' or str == 'Microsoft':
        return 'Microsoft'
    elif str == 'WeAreCisco' or str == 'Cisco' or str == 'ChuckRobbins':
        return 'Cisco'
    else:
        return 'Intel'

def show(str):
    links = re.findall(r'(https?://\S+)', str)
    if links:
        for i in links:
            #print(i)
            str = str.replace(i, '<a href=\'' + i + '\'>' + i + '</a>')
    str = '<li>' + str + '</li>'
    return str

In [11]:
for account in accounts:
    if account == '@PGelsinger':
        for status in tweepy.Cursor(api.user_timeline, screen_name=account, tweet_mode="extended", since="2021-02-15",).items():
            if isRelevant(clean(status.full_text)) and 'vmware' not in clean(status.full_text):
                #urls = []
                dt.append(status.created_at)
                source.append(status.user.screen_name)
                print(status.user.screen_name)
                try:
                    text.append('RT: ' + status.retweeted_status.full_text)
                except AttributeError:  # Not a Retweet
                    text.append(status.full_text)
#                 for i in range(len(status.entities['urls'])):
#                     urls.append(status.entities['urls'][i]['url'])
#                 links.append(' '.join(urls))
    else:     
        for status in tweepy.Cursor(api.user_timeline, screen_name=account, tweet_mode="extended", since="2020-09-01",).items():
            if isRelevant(clean(status.full_text)):
                #urls = []
                dt.append(status.created_at)
                source.append(status.user.screen_name)
                print(status.user.screen_name)
                try:
                    text.append('RT: ' + status.retweeted_status.full_text)
                except AttributeError:  # Not a Retweet
                    text.append(status.full_text)
#                 for i in range(len(status.entities['urls'])):
#                     urls.append(status.entities['urls'][i]['url'])
                #links.append(' '.join(urls))

satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella
satyanadella

jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfl

jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfluffy
jennylayfl

MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable

MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable
MSFTEnable

MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
MSFTnews
ChuckRobbins
ChuckR

WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel
WeAreIntel

In [27]:
df = pd.DataFrame(list(zip(dt, text, source)), columns = ['Date', 'Text', 'Source'])
df = df.drop_duplicates(subset=['Text'])
df.reset_index(inplace=True, drop=True)
df['Company'] = df['Source'].apply(lambda x:company(x))
df['Keywords'] = df['Text'].apply(lambda x: keys(clean(x)))
df['Labels'] = df['Text'].apply(lambda x: tags(clean(x)))
df['Show'] = df['Text'].apply(lambda x: show(x))
df

https://t.co/2PmIU1gshx
https://t.co/vEoE9upd4q
https://t.co/p9txm6ad56
https://t.co/SzRNC9UoUj
https://t.co/eKND5HCgDE
https://t.co/Pc619FAz4Z
https://t.co/1KixKqV5TY
https://t.co/qFuS6sFkM2
https://t.co/8He7Vdu2qk
https://t.co/dZgHWIgFJo
https://t.co/Lp4oj38Pcw
https://t.co/ID1eR1ayZd
https://t.co/Of6fytBaSO
https://t.co/d0BABNE8pl
https://t.co/sD1MUPfrzr
https://t.co/Qu6FcumNa8
https://t.co/L8HMOenoLz
https://t.co/m4bJE2ECyt
https://t.co/yN3x3ySnt7
https://t.co/bVWVojPV8K
https://t.co/WJlwTlBZF4
https://t.co/Q3xOeLPcwE
https://t.co/vvm5d8Og7k
https://t.co/JIKR1jxM1k
https://t.co/7y5g7OrcJ6
https://t.co/ryH6O4l5i9
https://t.co/6ulFTpeoCl
https://t.co/mfxlLUbVWB
https://t.co/9qG7aKIoZd
https://t.co/pMle5Gcmyx
https://t.co/VpV9T5Q2V2
https://t.co/S6o3sYfyOJ
https://t.co/WiIFK9uyZQ
https://t.co/8Y5o6dR4y2
https://t.co/084Bnj9KM8
https://t.co/iA8qzaKpZf
https://t.co/7yzGBQduvo
https://t.co/FliGJJ57Ly
https://t.co/7KCgKGdOjC
https://t.co/ZlknXzXUH7
https://t.co/HvepObCGJo
https://t.co/iGu

https://t.co/3aYvfBczQv
https://t.co/VdN36VManu.
https://t.co/TvEJTrEv68
https://t.co/rALt8ab5jx
https://t.co/lFxTtzFueI
https://t.co/sQCKhFHEau
https://t.co/fU1moMcCbb
https://t.co/sCnXhdTt9U
https://t.co/tdDpOgHv6n
https://t.co/3szYhUVnuf
https://t.co/a4SCkxrP3d
https://t.co/ySJCJcvfcU
https://t.co/VH9lzi3qyB
https://t.co/FgbhgROieP
https://t.co/GzmZlHprKF
https://t.co/7YDlGZmSO9
https://t.co/UtxGnoqab1
https://t.co/itIUFucJEc
https://t.co/H6H3QXvnwe
https://t.co/LKbLee1PtB
https://t.co/FQKcu1wckg
https://t.co/2HmysKwSuZ
https://t.co/NeczwYaxfN
https://t.co/FmDrhMMICt
https://t.co/xj6DKYMe8R
https://t.co/4n28EYXhJ8
https://t.co/AiisMokT0J
https://t.co/lwqZxRTQXX
https://t.co/d9rSoYGY3M
https://t.co/xqIV0gadLd
https://t.co/Ee5UQHy8Fg
https://t.co/bBowo5Oo00
https://t.co/czUCOxf6gk
https://t.co/4xm8UV1iix
https://t.co/MXxDmDcd4W
https://t.co/TxrIaz7i4i
https://t.co/VF9bkNbJeo
https://t.co/rxzHfQvTzV
https://t.co/c41xxmznJK
https://t.co/g7aj7dQ8eh
https://t.co/zbxorCaswL
https://t.co/lW

,Date,Text,Source,Company,Keywords,Labels,Show
0,2021-04-28 15:06:43,"Today, we are expanding our commitment to use ...",satyanadella,Microsoft,disabilities disability,disability,"<li>Today, we are expanding our commitment to ..."
1,2021-04-19 15:36:35,"Today, we announced our commitment to accelera...",satyanadella,Microsoft,inclusive jobs,culture hiring,"<li>Today, we announced our commitment to acce..."
2,2021-03-30 15:34:35,"To date, we’ve helped more than 30 million job...",satyanadella,Microsoft,job jobs seekers,hiring,"<li>To date, we’ve helped more than 30 million..."
3,2021-03-16 16:57:04,I’m looking forward to my conversation with @d...,satyanadella,Microsoft,culture diversity inclusion,culture,<li>I’m looking forward to my conversation wit...
4,2021-03-01 18:59:12,"Thank you, @MalaAnand_ for sharing this import...",satyanadella,Microsoft,abilities culture,culture disability,"<li>Thank you, @MalaAnand_ for sharing this im..."
...,...,...,...,...,...,...,...
4222,2018-06-21 09:44:27,"@kimhyun59411579 Hi, thanks for your interest!...",intel,Intel,jobs,hiring,"<li>@kimhyun59411579 Hi, thanks for your inter..."
4223,2018-06-17 05:12:57,@Tom_Sherry1 You can find and apply for all of...,intel,Intel,jobs,hiring,<li>@Tom_Sherry1 You can find and apply for al...
4224,2018-05-11 19:37:56,"@ramakrishnakalk Hello Rama Krishna, We hire g...",intel,Intel,hire,hiring,"<li>@ramakrishnakalk Hello Rama Krishna, We hi..."
4225,2018-03-28 19:00:00,Learn how new uses for #drones are sparking jo...,intel,Intel,job opportunities,hiring,<li>Learn how new uses for #drones are sparkin...


In [28]:
df['Labels'].value_counts()

disability                   2005
culture                       924
culture disability            725
hiring                        329
disability hiring             110
culture disability hiring      67
culture hiring                 45
                               22
Name: Labels, dtype: int64

In [29]:
df = df.loc[df.Labels != '']
df = df.loc[~((df.Labels == 'hiring') & (df.Keywords == 'interview'))]

In [ ]:
# df1 = df.loc[(df.Labels == 'disability')]
# df1.reset_index(drop=True, inplace=True)
# for i in range(len(df1)):
#     print(df1.Text[i])

In [ ]:
# df1

In [ ]:
# df1['Keywords'].value_counts()

In [ ]:
# df1 = df.loc[(df.Labels == 'disability') & (df.Keywords == 'accessibility')]
# df1.reset_index(drop=True, inplace=True)

In [ ]:
# for i in range(len(df1)):
#     print(df1.Text[i])

In [ ]:
# df1

In [ ]:
# df['Keywords'].unique()

In [ ]:
# df = df.loc[(df['Keywords'] != 'job') & (df['Keywords'] != 'employees employer employee') & (df['Keywords'] != 'employees employee')
#          & (df['Keywords'] != 'employee') & (df['Keywords'] != 'employment') & (df['Keywords'] != 'benefits') & (df['Keywords'] != 'hiring') &
#            (df['Keywords'] != 'benefits') & (df['Keywords'] != 'interview') & (df['Keywords'] != 'paralyzed') & (df['Keywords'] != 'hearing') & 
#             (df['Keywords'] != 'wheelchair') & (df['Keywords'] != 'disorder') & (df['Keywords'] != 'employment hiring') & (df['Keywords'] != 'representation')
#            & (df['Keywords'] != 'employment job') & (df['Keywords'] != 'employee job') & (df['Keywords'] != 'recruit') & (df['Keywords'] != 'paralyzed')]
# df.reset_index(inplace=True, drop=True)
# df

In [30]:
df['Company'].value_counts()

Microsoft    3131
Intel         680
Cisco         327
Name: Company, dtype: int64

In [ ]:
# inchiring = ['benefits','accommodate', 'accommodation', 'accomodating', 'interview', 'interviewing', 'employee benefits', 'health benefits', 
#              'perks', 'hire', 'hiring', 'job', 'careers', 'employment', 'recruit', 'equal employment', 'equal opportunity']
# empcomm = ['erg', 'employee resource group', 'resource group', 'disabled employee', 'disabled employees', 
#           'inclusivity', 'inclusion', 'inclusive',  'diversity', 'diverse', 'ndeam', 'national disability employment awareness month', 
#            'a11y', 'culture', 'dei', 'd&i']
        
# def inch(str):
#     for i in inchiring:
#         if re.search(r'\b' + i + r'\b', str.lower()):
#             return 1 
#     return 0
# def ecomm(str):
#     for i in empcomm:
#         if re.search(r'\b' + i + r'\b', str.lower()):
#             return 1 
#     return 0
# tabs = []
# for i in range(len(df)):
#     if df.Source[i] == 'satyanadella' or df.Source[i] == 'jennylayfluffy' or df.Source[i] == 'BradSmi' or df.Source[i] == 'ChuckRobbins' or df.Source[i] == 'PGelsinger' or df.Source[i] == 'AmyLWarner1':
#         tabs.append('Mission/Vision')
#     elif inch(df.Text[i]):
#         tabs.append('Inclusive Hiring')
#     elif ecomm(df.Text[i]):        
#         tabs.append('Employee Community')
#     else:
#         tabs.append('')
        
# #     elif df.Keywords[i].isin(inchiring):
# #         tabs.append('Inclusive Hiring')
# #     else:
# #         tabs.append('Employee Community')
# df['Tabs'] = tabs

In [ ]:
# def get_quote(str):
#     quote = re.findall('"[^"]*"', str)
#     people = []
#     for q in quote:
#         str = str.replace(q, '')
#     #print(str)
#     for i in nlp(str).ents:
#         if i.label_ == 'PERSON':
#             people.append(i.text)
#     return quote, people

In [ ]:
# df['Quote'] = df['Text'].apply(lambda x: get_quote(x)[0])
# df['People'] = df['Text'].apply(lambda x: get_quote(x)[1])

In [ ]:
# quote = '""There\'s a common saying in the disability community: You shouldn\'t build anything for us without including us." #Accessibility #technology https://t.co/Qiug7T4MY1'
# get_quote(quote)[0]

In [ ]:
# get_quote(quote)[1]

In [ ]:
# df

In [31]:
df.to_excel('Tweets.xlsx', header=True, index=False)

In [ ]:
# df = pd.read_excel('Tweets.xlsx')
# df

In [ ]:
# df1 = df.loc[df.Tabs =='Inclusive Hiring']
# df1['Company'].value_counts()

In [ ]:
# for i in df1.Text:
#     print(i)

In [32]:
df['Source'].value_counts()

MSFTEnable        1381
jennylayfluffy    1296
WeAreIntel         309
BradSmi            275
MSFTnews           202
Cisco              155
WeAreCisco          96
Microsoft           90
PGelsinger          90
satyanadella        89
ChuckRobbins        76
AmyLWarner1         43
intel               36
Name: Source, dtype: int64

In [33]:
inclusive_hiring = ['hiring', 'disability hiring', 'culture hiring', 'culture disability hiring']
employee_community = ['culture', 'culture disability', 'culture hiring', 'culture disability hiring']

In [34]:
inc_hiring = df.loc[df['Labels'].isin(inclusive_hiring)]
inc_hiring = inc_hiring.loc[(df['Source'] != 'satyanadella') & (df['Source'] != 'jennylayfluffy') & (df['Source'] != 'BradSmi')
                   & (df['Source'] != 'PGelsinger') & (df['Source'] != 'ChuckRobbins') & (df['Source'] != 'AmyLWarner1')]
inc_hiring.to_excel('Inclusive Hiring.xlsx', header=True, index=False)
emp_community = df.loc[df['Labels'].isin(employee_community)]
emp_community = emp_community.loc[(df['Source'] != 'satyanadella') & (df['Source'] != 'jennylayfluffy') & (df['Source'] != 'BradSmi')
                   & (df['Source'] != 'PGelsinger') & (df['Source'] != 'ChuckRobbins') & (df['Source'] != 'AmyLWarner1')]
emp_community.to_excel('Employee Community.xlsx', header=True, index=False)

In [35]:
inc_hiring

,Date,Text,Source,Company,Keywords,Labels,Show
1686,2021-04-09 22:00:01,LinkedIn career expert Catherine Fisher explai...,Microsoft,Microsoft,inclusive job seekers,culture hiring,<li>LinkedIn career expert Catherine Fisher ex...
1693,2021-03-25 18:10:24,@mlaw91 @MSPowerApps Consider Floofy hired. 😻,Microsoft,Microsoft,hired,hiring,<li>@mlaw91 @MSPowerApps Consider Floofy hired...
1736,2021-03-02 16:36:46,We need to create continuous feedback loops be...,Microsoft,Microsoft,jobs,hiring,<li>We need to create continuous feedback loop...
1742,2021-02-25 00:26:01,RT: Inclusive hiring programs empower candidat...,Microsoft,Microsoft,culture disabilities hiring inclusive,culture disability hiring,<li>RT: Inclusive hiring programs empower cand...
1752,2021-02-09 19:45:00,Tech-oriented jobs are expected to increase ne...,Microsoft,Microsoft,jobs,hiring,<li>Tech-oriented jobs are expected to increas...
...,...,...,...,...,...,...,...
4220,2018-09-07 23:00:12,What are the new jobs in a human + machine wor...,intel,Intel,jobs,hiring,<li>What are the new jobs in a human + machine...
4222,2018-06-21 09:44:27,"@kimhyun59411579 Hi, thanks for your interest!...",intel,Intel,jobs,hiring,"<li>@kimhyun59411579 Hi, thanks for your inter..."
4223,2018-06-17 05:12:57,@Tom_Sherry1 You can find and apply for all of...,intel,Intel,jobs,hiring,<li>@Tom_Sherry1 You can find and apply for al...
4224,2018-05-11 19:37:56,"@ramakrishnakalk Hello Rama Krishna, We hire g...",intel,Intel,hire,hiring,"<li>@ramakrishnakalk Hello Rama Krishna, We hi..."


In [37]:
inc_hiring['Company'].value_counts()

Intel        94
Microsoft    71
Cisco        59
Name: Company, dtype: int64

In [25]:
emp_community

,Date,Text,Source,Company,Keywords,Labels,Show
1678,2021-05-12 18:00:00,Geography shouldn't limit possibility.\n\nThat...,Microsoft,Intel,diversity,culture,<li>Geography shouldn't limit possibility.\n\n...
1681,2021-04-26 18:00:02,"When technology is inclusive, everyone benefit...",Microsoft,Intel,accessibility inclusive,culture disability,"<li>When technology is inclusive, everyone ben..."
1682,2021-04-23 21:00:00,The number of women in technical roles at Micr...,Microsoft,Intel,diversity inclusion,culture,<li>The number of women in technical roles at ...
1683,2021-04-20 00:07:00,RT: Join us to reimagine the future of disabil...,Microsoft,Intel,accessibility disability inclusion,culture disability,<li>RT: Join us to reimagine the future of dis...
1684,2021-04-19 22:03:14,RT: We continue to grow our cloud infrastructu...,Microsoft,Intel,inclusive,culture,<li>RT: We continue to grow our cloud infrastr...
...,...,...,...,...,...,...,...
4210,2019-02-08 17:08:10,RT: Intel’s @CGoughPDX challenges industry lea...,intel,Intel,culture,culture,<li>RT: Intel’s @CGoughPDX challenges industry...
4211,2019-01-22 23:00:11,"With a diverse worldwide workforce, closing th...",intel,Intel,diverse,culture,"<li>With a diverse worldwide workforce, closin..."
4216,2018-10-29 16:18:35,RT: Intel has achieved full representation in ...,intel,Intel,representation,culture,<li>RT: Intel has achieved full representation...
4221,2018-07-24 16:16:55,Today @Forbes recognized Intel as one of the b...,intel,Intel,diversity,culture,<li>Today @Forbes recognized Intel as one of t...


In [38]:
emp_community['Company'].value_counts()

Microsoft    483
Intel        373
Cisco        180
Name: Company, dtype: int64

In [ ]:
# df['Tabs'].value_counts()

In [ ]:
# df['Quote'].value_counts()